# DQN (Deep Q-Network)

***

<br/>

## Deep Q-Network(DQN)이란?

Google Deepmind에서 개발한 알고리즘으로, 강화학습에 Deep Learning을 적용하는 방식

<br/>

### Q Learning

강화학습은 Reward(보상)을 통해 학습하는 방식으로, 매 순간 보상이 가장 높다고 판단되는 행동을 취한다.

Q Learning은 모든 가능한 상태-행동 조합을 Table로 그리고 거기에 모든 Q-Value를 적어서 사용한다. 그 후에는 벨만 방정식을 이용해서 표를 업데이트한다.

<br/>

<img src="img/0.png" width='500'>

<br/>

### The Bellman Equation(벨만 방정식), Q-value(Quality Value)

모든 상태에서 추하는 각각의 행동에 대해 보상을 얼마나 받을지 알고있다고 가정하면 우리는 가장 높은 보상을 받을 수 있는 행동들을 연속적으로 할 수 있다. 이렇게 최종적으로 받는 모든 보상의 총합을 Q-value라고 한다.

벨만 방정식이란 현재 상태의 가치함수와 다음 상태의 가치함수 사이의 관계식이다.

$$
Q(s, a) = r(s, a) + \gamma \max_aQ(s', a)
$$

$Q(s, a)$: 상태 $s$에서 행동 $a$를 취할 때의 $Q$ 값, 즉 행동 가치 (action-value)

$r(s,a)$: 상태 $s$에서 행동 $a$를 취했을 때 얻는 즉각적인 보상 (immediate reward)

$\gamma$: 할인 계수 (discount factor)로서 미래 보상에 대한 가중치를 조절하는 요소. 이 값이 작을수록 현재 보상을 비래에 덜 중요하게 여긴다.

$\max_aQ(s', a)$: 다음 상태 $s'$에서 가능한 모든 행동 중에서 최대 $Q$ 값

이 수식을 해석하면 어떠한 행동을 했을 때 받는 보상($r(s, a)$)과 다음 상태에서 가능한 모든 행동중에서 최적의 행동(미래 보상,$\gamma \max_aQ(s', a)$)의 합을 행동가치($Q(s, a)$)에 할당한다.

### Deep Q-Network(DQN)

DQN은  가치  이터레이션(Value  iteration)을  바탕으로 한 학습법이다.  알고리즘은 벨만 상태 가지 방정식(Bellman  equation)을 만족하는 Q값을 찾는 것을 목표로 하고 작동한다.
다시말해 Deep Q-Network는 Q-Learning과 Deep Learning을 합친 방식이라고 말할 수 있다.
DQN은 이러한 Q-가치 함수를 딥 뉴럴 네트워크로 근사화하고, 경험 재생(Experience Replay) 및 고정된 목표 네트워크(Fixed Target Network)를 사용하여 안정적인 학습을 수행한다.
이 모델에 대한 표현은 $Q(s, a; \theta)$이다.($\theta$: 학습 가중치)


## DQN CartPole 예제

CartPole는 마찰이 없는 트랙에 막대기(pole)가 연결 되어있는 카트(cart)를 움직여 막대기를 넘어지지 않도록 하는 게임이다.

에피소드가 종료되는 조건 은 아래와 같다.

* 막대기가 수직으로부터 12도 이상 기울어짐 (-12도 ~ 12도).
* 카트가 중심으로부터 2.4 이상 벗어남 (-2.4 ~ 2.4).
* 시간 스텝이 200보다 커짐 (CartPole-v1의 경우 500).

이 게임을 수행하는 프로그램을 DQN으로 학습시켜 구현하는것이 이 예제의 목표이다.

<br/>

<img src="img/1.png" width='500'>

### 0. gym 설치

Gym은 강화학습 알고리즘을 개발하고 비교하기 위한 툴킷이다. agent의 구조에 대해서 어떠한 가정을 하지 않으며, TensorFlow와 Theano와 같은 라이브러리와 호환 가능하다.

Gym 라이브러리는 우리의 강화학습 알고리즘을 적용할 테스트 문제 (환경)들의 모음이다. 이러한 환경들은 인터페이스를 공유하며, 일반적인 알고리즘을 시도할 수 있도록 해준다.

처음에 gym 0.26버전을 사용해봤는데 알 수 없는 오류가 발생하여 0.21버전을 사용했다.

0.21버전을 사용했을 때 `NameError: name 'glPushMatrix' is not defined` 에러가 발생하여 s[tackoverflow.com에서 찾아본 결과](https://stackoverflow.com/questions/74314778/nameerror-name-glpushmatrix-is-not-defined) pyglet을 설치하면 해결이 된다는 글을 보고 pyglet을 설치하였다.

In [5]:
!pip3 install gym==0.21
!pip3 install pyglet==1.5.27

### 1. 필요한 라이브러리 및 환경 로드

필요한 라이브러리와 환경을 불러온다. Gym은 환경을 제공하는 라이브러리이며, TensorFlow 및 Keras는 딥 러닝 모델을 구축하는데 사용된다.

In [6]:
import gym
import numpy as np
import tensorflow as tf
from keras.optimizers import Adam
from keras import Sequential
from keras.layers import Input, Dense
import random

### 2. CartPole 환경 로드

Gym을 사용하여 "CartPole-v1" 환경을 로드하고, 가능한 행동의 수를 가져온다.

In [7]:
# CartPole 환경 로드
env = gym.make("CartPole-v1")
num_actions = env.action_space.n

### 3. 사용자 입력 받기

사용자로부터 "Y" 또는 "N"을 입력받아서 기존에 학습된 모델을 사용할지 여부를 결정한다.

In [8]:
# 사용자 입력 받기
user_input = input("학습된 모델을 사용하시겠습니까? (Y/N): ")

### 4. 모델 생성 또는 불러오기

사용자 입력에 따라서 새로운 모델을 생성하거나 기존에 학습된 모델을 불러온다.

In [9]:
if user_input.lower() == "y":
    # 기존에 학습된 모델을 불러오기
    model = tf.keras.models.load_model("dqn_model.h5")
else:
    # Q-Network 모델 생성
    model = Sequential([
        Input(shape=(4,)),
        Dense(32, activation="relu"),
        Dense(64, activation="relu"),
        Dense(num_actions)
    ])

2023-09-12 16:10:55.954180: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M1 Pro
2023-09-12 16:10:55.954201: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 32.00 GB
2023-09-12 16:10:55.954206: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 10.67 GB
2023-09-12 16:10:55.954242: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:303] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-09-12 16:10:55.954256: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:269] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


### 5. 모델 컴파일 및 학습 매개변수 설정

In [10]:
    # 최적화 알고리즘 및 손실 함수 설정
    optimizer = Adam(learning_rate=0.001)
    loss_fn = tf.losses.mean_squared_error

    # 모델 컴파일
    model.compile(optimizer=optimizer, loss=loss_fn)

    # 학습 매개변수 설정
    epsilon = 1.0
    epsilon_min = 0.1
    epsilon_decay = 0.995
    batch_size = 32
    gamma = 0.99

모델을 컴파일하고, 학습에 필요한 하이퍼파라미터를 설정한다.

1. `optimizer = Adam(learning_rate=0.001)`:
> 이 코드는 Adam 최적화 알고리즘을 초기화하고 학습률(learning rate)을 0.001로 설정한다.

2. `loss_fn = tf.losses.mean_squared_error`:
> 이 부분은 손실 함수(loss function)를 지정한다.
> DQN에서는 주로 평균 제곱 오차(mean squared error)를 사용한다.
> 모델의 예측 Q-값과 실제 Q-값 사이의 차이를 최소화하기 위해 이 손실 함수를 사용한다.

3. `model.compile(optimizer=optimizer, loss=loss_fn)`:
> 이 코드는 모델을 컴파일한다. 컴파일 단계에서는 최적화 알고리즘과 손실 함수를 모델에 연결하여 학습 프로세스를 설정한다.

4. `epsilon = 1.0`:
> 이는 탐험(exploration) 비율을 나타내며, 초기에는 높게 설정된다.
> 탐험 비율은 에이전트가 무작위로 행동을 취할 가능성을 나타낸다.

5. `epsilon_min = 0.1`:
> 최소 탐험 비율을 나타낸다.
> 탐험 비율은 학습이 진행됨에 따라 점차 감소하며, 이 최소값보다 낮아지지 않는다.

6. `epsilon_decay = 0.995`:
> 탐험 비율 감소율을 나타낸다.
> 매 에피소드(또는 타임 스텝)마다 탐험 비율이 감소되어 무작위 탐험을 줄이고, 학습된 정책에 더 가까운 행동을 선택하게 된다.

7. `batch_size = 32`:
> 미니배치 크기를 나타낸다.
> 경험 재생(Experience Replay)에서 사용되며, 학습 데이터를 작은 미니배치로 나누어 모델을 업데이트한다.
> 작은 미니배치를 사용함으로써 학습이 안정화되고 메모리 효율성이 향상된다.

8. `gamma = 0.99`:
> 할인 계수(discount factor)로, 미래 보상을 현재 보상에 대한 가중치로 고려하는 역할을 한다.
> 더 높은 값은 미래 보상을 더 중요하게 다룬다.

6. Experience Replay를 위한 데이터 저장소 초기화:

에피소드 데이터를 저장하기 위한 데이터 저장소를 초기화합니다.

In [11]:
    # Experience Replay를 위한 데이터 저장소 초기화
    replay_memory = []

7. DQN 학습:
주어진 에피소드 수에 따라 DQN을 학습한다.
에피소드마다 환경을 초기화하고, epsilon-greedy 정책을 따라 행동 선택하고, 환경에서 행동을 실행하며 데이터를 수집하고, Experience Replay 및 모델 업데이트를 수행한다.
학습이 진행됨에 따라 epsilon 값을 감소시켜 더 많은 탐험을 하도록 한다.

In [ ]:
    # DQN 학습
    num_episodes = 1000
    for episode in range(num_episodes):
        state = env.reset()
        total_reward = 0

        while True:
            if np.random.rand() <= epsilon:
                action = env.action_space.sample()
            else:
                q_values = model.predict(np.reshape(state, [1, 4]))
                action = np.argmax(q_values[0])

            # 환경에서 행동 실행 및 다음 상태 및 보상 얻기
            step_result = env.step(action)
            next_state, reward, done, info = step_result

            if done:
                reward = -10

            replay_memory.append((state, action, reward, next_state, done))

            if len(replay_memory) >= batch_size:
                minibatch = random.sample(replay_memory, batch_size)
                for state, action, reward, next_state, done in minibatch:
                    target = reward
                    if not done:
                        target += gamma * np.amax(model.predict(np.reshape(next_state, [1, 4]))[0])
                    target_f = model.predict(np.reshape(state, [1, 4]))
                    target_f[0][action] = target
                    model.fit(np.reshape(state, [1, 4]), target_f, epochs=1, verbose=0)

            state = next_state
            total_reward += reward

            if done:
                break

        if epsilon > epsilon_min:
            epsilon *= epsilon_decay

        print(f"Episode: {episode + 1}, Total Reward: {total_reward}")

### 8. 학습된 모델 저장

In [13]:
    # 학습된 모델 저장
    model.save("dqn_model.h5")

/opt/homebrew/Caskroom/miniforge/base/envs/tensorflow-dev/lib/python3.8/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


### 9. 학습된 모델을 사용하여 CartPole 실행

학습된 모델을 사용하여 CartPole을 실행하고, 에피소드의 총 보상을 확인한다.
 실행 과정을 시각화하기 위해 env.render()를 사용한다.

In [14]:
# 학습된 모델을 사용하여 CartPole을 실행하여 결과 확인
for _ in range(5):
    state = env.reset()
    total_reward = 0
    while True:
        action = np.argmax(model.predict(np.reshape(state, [1, 4]))[0])
        next_state, reward, done, _ = env.step(action)
        total_reward += reward
        state = next_state
        env.render()
        if done:
            break

env.close()

1/1 [==============================] - 0s 11ms/step


### Reference

> [https://engineering-ladder.tistory.com/68](https://engineering-ladder.tistory.com/68)
> [https://jeinalog.tistory.com/20](https://jeinalog.tistory.com/20)
> [https://myetc.tistory.com/36](https://myetc.tistory.com/36)
> Yoonchae Kim and Dong-min Park, "Experiment and Analysis of Learning Rate of DQN for solving Cartpole Problem," in 한국정보과학회 학술발표논문집, 2022, pp. 1889-1891.